In [1]:
#from lib.project_5 import load_data_from_database, make_data_dict, general_model, general_transformer

# Step 3 - Build Model

**NOTE: EACH OF THESE SHOULD BE WRITTEN SOLELY WITH REGARD TO STEP 3 - Build Model**

### Domain and Data

**TODO:** Write a simple statement about the domain of your problem and the dataset upon which you will be working. 
**We ran Logistic Regression on Madelone data set. We also reviewed coefficients for each feature from 500 features and identified salient features. Below are the salient feature we idenfied:**



![](http://localhost:8888/notebooks/dsi/dsi-workspace/project-05/images/salient_features_step2.png)

### Problem Statement

We seek to idenfity features from Madelon data set that affect out target i.e. Label. There are 500 features given to us our goal is identify salient features. We do not know if features identifed above are the ones affecting our target so we like to compare our model with other models. 


### Solution Statement

We will build model and compare that against different models we will predict the results based on best model and provide a report.

### Metric

We will use SelectKBest, KNN, GridSearchCV and LogisticRegression and review and compare score and features.

### Benchmark

**TODO**: This should refer to Step 1.

In [1]:
from sqlalchemy import create_engine
import pandas as pd
#from os import chdir; chdir('..')
from os import chdir; chdir('./lib')
from sklearn.preprocessing import StandardScaler
from project_5 import load_data_from_database, make_data_dict,general_transformer, general_model
from sklearn import linear_model
#from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier


In [2]:
!pwd

/src/dsi/dsi-workspace/project-05/lib


** Load Data, define X and y, Split data in to train and test, and Scale the data for analysis. We will split data in to 50% train and 50% test **

In [3]:
madelon_df = load_data_from_database()
del madelon_df["index"]
# Define X and y
y = madelon_df["label"]
X = madelon_df.drop("label",axis=1)


** Construct a pipeline that uses LogisticRegression **
1. Build a new pipeline for LogisticRegression.
2. Ran model on L1 and L2
3. With Regularization 0.01, .1, .2, 0.03
4. Run 5 Fold Grid Serach
5. Review the Score

In [4]:
data_dict= make_data_dict(X,y,0.5,random_state=42)
data_dict = general_transformer(StandardScaler(),data_dict)
lr_model = LogisticRegression()
lr_params = {
    'C' : [1E-2, 1E-1, 2E-1, 3E-2],
    'penalty' : ['l1', 'l2']
}
gs_model = GridSearchCV(lr_model, param_grid=lr_params, cv=5)
data_dict = general_model(gs_model, data_dict)
data_dict["processes"]

[StandardScaler(copy=True, with_mean=True, with_std=True),
 GridSearchCV(cv=5, error_score='raise',
        estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
        fit_params={}, iid=True, n_jobs=1,
        param_grid={'penalty': ['l1', 'l2'], 'C': [0.01, 0.1, 0.2, 0.03]},
        pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
        scoring=None, verbose=0)]

In [6]:
gs_model1 = data_dict["processes"][1]

In [7]:
print gs_model1.best_params_
print gs_model1.best_score_
print gs_model1.best_estimator_

{'penalty': 'l1', 'C': 0.03}
0.626
LogisticRegression(C=0.03, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


### Of all models LogisticRegression with L1 with C value 0.03 had highest test score of 62.6%

In [28]:
lr_result_df = pd.DataFrame(gs_model1.cv_results_)

In [29]:
lr_result_df

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_penalty,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.010505,0.000558,0.503,0.503000,0.01,l1,"{u'penalty': u'l1', u'C': 0.01}",8,0.502488,0.503129,...,0.505,0.50250,0.502513,0.503121,0.502513,0.503121,0.002427,0.000263,0.001000,0.000250
1,0.030319,0.000495,0.576,0.877494,0.01,l2,"{u'penalty': u'l2', u'C': 0.01}",3,0.577114,0.883605,...,0.640,0.85750,0.537688,0.887640,0.502513,0.888889,0.002865,0.000214,0.051083,0.012071
2,0.075427,0.000405,0.581,0.816245,0.1,l1,"{u'penalty': u'l1', u'C': 0.1}",2,0.572139,0.814768,...,0.625,0.80750,0.537688,0.818976,0.557789,0.827715,0.031537,0.000013,0.032801,0.006834
3,0.069390,0.000393,0.554,0.955741,0.1,l2,"{u'penalty': u'l2', u'C': 0.1}",6,0.567164,0.961202,...,0.610,0.94875,0.512563,0.971286,0.502513,0.962547,0.007483,0.000002,0.040480,0.012651
4,0.094846,0.000465,0.572,0.882993,0.2,l1,"{u'penalty': u'l1', u'C': 0.2}",4,0.597015,0.884856,...,0.630,0.88125,0.517588,0.891386,0.532663,0.890137,0.020145,0.000081,0.041426,0.008639
5,0.076968,0.000400,0.545,0.977993,0.2,l2,"{u'penalty': u'l2', u'C': 0.2}",7,0.557214,0.982478,...,0.580,0.97125,0.517588,0.983770,0.502513,0.990012,0.001161,0.000007,0.029707,0.009848
6,0.021284,0.000415,0.626,0.648740,0.03,l1,"{u'penalty': u'l1', u'C': 0.03}",1,0.616915,0.642053,...,0.630,0.64125,0.613065,0.677903,0.628141,0.642946,0.008285,0.000028,0.010204,0.014624
7,0.043668,0.000395,0.559,0.915991,0.03,l2,"{u'penalty': u'l2', u'C': 0.03}",5,0.601990,0.921151,...,0.600,0.90500,0.527638,0.923845,0.482412,0.931336,0.000484,0.000009,0.046667,0.012217


**Construct a pipeline that uses SelectKBest, KNN**
1. Build a new pipeline for SelectKBest and KNN.
2. Set the neighbors between 11 and 21
3. Select the max of 10 features using SelectKBest
4. Run 5 Fold Grid Serach
5. Review the Score
6. Compare with LogisticRegression Score
7. Review features

In [8]:
data_dict1= make_data_dict(X,y,0.5,random_state=42)
data_dict1 = general_transformer(SelectKBest(f_classif,k=10),data_dict1)
KNN_model = KNeighborsClassifier(n_neighbors=10)
KNN_params = {
    'n_neighbors': [11,13,15,17,19,21]
}
gs_model = GridSearchCV(KNN_model, param_grid=KNN_params, cv=5)
data_dict1 = general_model(gs_model, data_dict1)
data_dict1["processes"]

[SelectKBest(k=10, score_func=<function f_classif at 0x7fe14a4cc938>),
 GridSearchCV(cv=5, error_score='raise',
        estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=10, p=2,
            weights='uniform'),
        fit_params={}, iid=True, n_jobs=1,
        param_grid={'n_neighbors': [11, 13, 15, 17, 19, 21]},
        pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
        scoring=None, verbose=0)]

In [11]:
gs_model2 = data_dict1["processes"][1]

In [12]:
print gs_model2.best_params_
print gs_model2.best_score_
print gs_model2.best_estimator_

{'n_neighbors': 11}
0.816
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=11, p=2,
           weights='uniform')


In [18]:
KBest = data_dict1["processes"][0]

In [19]:
features_index = KBest.get_support(indices=True)

In [26]:
features_index

array([ 48,  64, 105, 128, 241, 336, 338, 378, 442, 475])

In [30]:
KNN_result_df = pd.DataFrame(gs_model2.cv_results_)

In [31]:
KNN_result_df

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_n_neighbors,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.001510,0.003474,0.816,0.845510,11,{u'n_neighbors': 11},1,0.815920,0.854819,0.805970,...,0.780,0.84500,0.844221,0.837703,0.834171,0.832709,0.000765,0.001045,0.022441,0.009500
1,0.001096,0.002569,0.806,0.835759,13,{u'n_neighbors': 13},2,0.791045,0.843554,0.791045,...,0.790,0.82875,0.819095,0.830212,0.839196,0.827715,0.000067,0.000051,0.019863,0.008594
2,0.001050,0.002640,0.796,0.829507,15,{u'n_neighbors': 15},3,0.776119,0.834793,0.776119,...,0.770,0.82625,0.819095,0.823970,0.839196,0.822722,0.000019,0.000058,0.027777,0.006651
3,0.001129,0.002739,0.795,0.816509,17,{u'n_neighbors': 17},4,0.776119,0.819775,0.766169,...,0.785,0.81500,0.829146,0.816479,0.819095,0.800250,0.000095,0.000077,0.024623,0.009884
4,0.001054,0.002918,0.791,0.813254,19,{u'n_neighbors': 19},5,0.781095,0.818523,0.786070,...,0.770,0.81375,0.819095,0.812734,0.798995,0.805243,0.000013,0.000171,0.016811,0.004474
5,0.001105,0.002872,0.787,0.807751,21,{u'n_neighbors': 21},6,0.761194,0.814768,0.791045,...,0.770,0.80125,0.814070,0.811486,0.798995,0.805243,0.000099,0.000046,0.019220,0.004791


### We received 83% score with  K Nearest Neighbors with SelectKBest transformer. Which is a higher score compare to Logistic Regression and we will go with this model. Feature Selected by this model are shown above.

## Implementation

Implement the following code pipeline using the functions you write in `lib/project_5.py`.

<img src="assets/build_model.png" width="600px">